In [1]:
import os
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import TensorDataset, DataLoader, Dataset

from utils.util import *

In [3]:
DATA_PATH = '/home/ralleking/Code/Python/Datasets/meastro'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## pre-processing

In [5]:
folds = read_txt_files(DATA_PATH)

In [24]:
train = np.array(folds[2])
train_labels = np.array(folds[3])

#column order, predict only one timestep
train_labels = train_labels[:,:1]
train_labels = train_labels[:,0,:]
train = np.transpose(train, (0,2,1))

train_dataset = TensorDataset(torch.from_numpy(train).float().to(DEVICE), torch.from_numpy(train_labels).float().to(DEVICE))

## model

In [7]:
class Baseline(nn.Module):
    """
    Simple baseline lstm model
    """

    def __init__(self, time_steps, hidden_dim, num_classes):

        super(Baseline, self).__init__()
        self.hidden_dim = hidden_dim

        #lstm layer
        self.lstm = nn.LSTM(time_steps, hidden_dim)

        #output layer
        self.linear = nn.Linear(hidden_dim, num_classes)
        

        #dropout
        self.do = nn.Dropout(0.6)

    def forward(self, input):
        lstm_out, _ = self.lstm(input.view(len(input), 1, -1))
        out = self.do(lstm_out)
        predictions = self.linear(out.view(len(input), -1))
        return predictions[-1]


## main

In [26]:
time_steps = 100
num_classes = 15

In [28]:
#model definitions
model = Baseline(time_steps, 1, num_classes)
model.to(DEVICE)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 4

for i in range(epochs):
    for x, y in train_dataset:
        optimizer.zero_grad()
        y_pred = model(x)
        single_loss = loss_function(y_pred, y)
        single_loss.backward()
        optimizer.step()
    print(single_loss)

tensor(105.8584, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(61.2478, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(76.7213, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(46.4304, device='cuda:0', grad_fn=<MseLossBackward>)


In [29]:
#torch.save(model, '/home/ralleking/Code/Python/models/basline_model.pt')
loaded_model = torch.load('/home/ralleking/Code/Python/models/basline_model.pt')

In [168]:
def predict_next_note(data, model):
    #prediction
    pred = model(data)
    #add dim for cat
    pred = pred.view((15,1))
    #concatenate data and pred
    data = torch.cat([data, pred], dim=1)
    #drop first row
    data = data[:,1:]    
    return data

def to_midi(data, save_path, midi_name='test.mid'):
    #split top and bottom of data (note info/pitchclasses)
    top = data[0:3,:]
    pitch_class_preds = data[3:,:]
    #make pitchclass one hot
    temp = np.zeros(pitch_class_preds.shape)
    temp[np.argmax(pitch_class_preds, axis=0)] = 1
    #round top matrix
    top = np.round(top, 0)
    #combine again
    data = np.vstack([top, temp])
    data = np.array(data, dtype='int32')
    print(type(data))
#    array_to_midi(data, save_path, midi_name)

In [179]:
seq = train_dataset[0][0]
pred_len = 100
save_path = '/home/ralleking/Code/Python/models/'
for i in range(0, pred_len):
    seq = predict_next_note(seq, loaded_model)
    
data = seq.to("cpu").detach().numpy()

array_to_midi(train[1], save_path, 'midi_name')

to_midi(data, save_path)

IndexError: invalid index to scalar variable.

TypeError: type numpy.ndarray doesn't define __round__ method